<a href="https://colab.research.google.com/github/viswavich/REACT-INTERN-TASK-2/blob/main/TAMIL_QWEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install accelerate gTTS gradio transformers torchaudio openai-whisper

In [8]:
pip install gradio gTTS transformers accelerate torch openai-whisper

In [9]:
!pip install --upgrade gradio

In [10]:
!pip install --upgrade gradio
!pip install gTTS transformers accelerate torch openai-whisper

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from gtts import gTTS
import torch
import gradio as gr
import whisper

# Load Whisper model
whisper_model = whisper.load_model("base")

# CUDA check
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Qwen model
language_model_name = "Qwen/Qwen2-1.5B-Instruct"
language_model = AutoModelForCausalLM.from_pretrained(
    language_model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(language_model_name)

# Qwen prompt
system_prompt = "You are an AI assistant.\ If the user speaks in Tamil,\ respond in Tamil."

# Chat function
def process_input(input_text):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": input_text}
    ]
    chat_prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    model_inputs = tokenizer([chat_prompt], return_tensors="pt").to(device)

    generated_ids = language_model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    output_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return output_text

# Text to Tamil speech
def text_to_speech(text):
    tts = gTTS(text=text, lang="ta")
    filename = "output_tamil.mp3"
    tts.save(filename)
    return filename

# Main function
def handle_voice_input(audio_file):
    transcription = whisper_model.transcribe(audio_file, language="ta")["text"]
    reply_text = process_input(transcription)
    audio_reply = text_to_speech(reply_text)
    return transcription, reply_text, audio_reply

# Gradio Interface
with gr.Blocks(title="Tamil Voice Chatbot") as demo:
    gr.Markdown("## 🎤 Tamil Voice Chatbot")
    gr.Markdown("Upload/Record Tamil audio to transcribe and reply in Tamil")

    with gr.Row():
        mic_input = gr.Audio(type="filepath", label="🎙️ Upload or Record Tamil Audio")

    with gr.Row():
        transcribed = gr.Textbox(label="📝 Transcription (Tamil)")
        response = gr.Textbox(label="🤖 Qwen's Reply in Tamil")

    audio_out = gr.Audio(label="🔊 Spoken Tamil Reply")

    btn = gr.Button("Generate Reply")

    btn.click(fn=handle_voice_input, inputs=[mic_input], outputs=[transcribed, response, audio_out])

# Launch app
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cdad212dcea719cff4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
